In [3]:
# 연습문제 9-5. 인터파크 쇼핑몰의 Best seller 상품 정보 추출하기

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver

import time
import sys
import re
import math
import numpy  
import pandas as pd 
import xlwt 
import random
import os

import urllib.request
import urllib

import win32com.client as win32   #pywin32 설치후 동작
import win32api  #pypiwin32 설치후 동작 ,파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
                 #파이썬 쉘을 관리자 권한으로 실행한 후 불러오기로 이 소스 실행하기

#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.

print("=" *80)
print("연습문제 9-5. 인터파크 쇼핑몰의 Best seller 상품 정보 추출하기")
print("=" *80)

query_txt = '인터파크쇼핑'
query_url='http://shopping.interpark.com/best/main.do?&smid1=s_menu&smid2=best'

cnt = int(input('  1.크롤링 할 건수는 몇건입니까?: '))
real_cnt = math.ceil(cnt / 20)


f_dir=input('  2.파일이 저장될 경로만 쓰세요;(예:c:\\temp\\): ')
print("\n")

if cnt > 30 :
      print("  요청 건수가 많아서 시간이 제법 소요되오니 잠시만 기다려 주세요~~")
else :
      print("  요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요~~")
      
# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

img_dir = f_dir+s+'-'+query_txt+"\\images"

os.makedirs(img_dir)

os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

s_time = time.time( )

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get(query_url)
time.sleep(2)

#화면 스크롤을 위해 스크롤 함수를 만들고 실행합니다
def scroll_down(driver):
      driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
      time.sleep(1)


i = 1
while (i <= 3):
         scroll_down(driver) 
         i += 1

ranking2=[]
title2=[]
o_price2=[]
s_price2=[]
benefit2=[]


html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
        
count = 1

sale_result = soup.find('div','recommendList')
slist = sale_result.select('ul > li')

#이미지 추출하기
img_file_no = 0

for li in slist:
                
            os.chdir(img_dir)

            try :
                  photo = li.find('div','imgBox').find('img')['src']
            except AttributeError :
                continue
            
            img_file_no += 1

            urllib.request.urlretrieve(photo,str(img_file_no)+'.jpg')
            time.sleep(1)

            if img_file_no > cnt :
                break

              
#텍스트 추출하기

for li in slist:
            
            f = open(ff_name, 'a',encoding='UTF-8')
            f.write("-----------------------------------------------------"+"\n")

            print("\n")
            print("-" *70)
            
            try :
             ranking = li.find('h3').get_text()
            except AttributeError :
             ranking = ''
             print('1.판매순위:',ranking.replace("\n",""))
            else :
             print("1.판매순위:",ranking)

             f.write('1.판매순위:'+ ranking + "\n")
             

            try :
             title = li.find('div','infoBox').find('a').get_text()
            except AttributeError :
             title = ''
             print(title)
             f.write('2.제품소개:'+ title + "\n")
            else :
             print("2.제품소개:", title.replace("\n",""))
             f.write('2.제품소개:'+ title + "\n")
            

             s_price = li.find('div', class_='price').get_text().replace("\t","").replace("\n","")
             print("3.판매가격:", s_price)
             f.write('3.판매가격:'+ s_price + "\n")

             try :
               benefit = li.find('div', class_='benefit').get_text().replace("\n","")
             except AttributeError:
               benefit = ' '
             print("4.혜택:", benefit)
             f.write('4.혜택:'+ benefit + "\n")


             ranking2.append(ranking)
             title2.append(title.replace("\n",""))
             s_price2.append(s_price.replace("\n",""))
             benefit2.append(benefit.replace("\n",""))

             if count == cnt :
               break

             count += 1

             time.sleep(0.3)

#step 9. 엑셀 형태로 저장하기
              
i_best_seller = pd.DataFrame()

i_best_seller['판매순위']=ranking2
i_best_seller['제품소개']=pd.Series(title2)
i_best_seller['판매가격']=pd.Series(s_price2)
i_best_seller['혜택']=pd.Series(benefit2)

# csv 형태로 저장하기
i_best_seller.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
i_best_seller.to_excel(fx_name ,index=True)



# 요약 정보 출력하기

e_time = time.time( )
t_time = e_time - s_time

# txt 파일에 요약정보 저장하기
orig_stdout = sys.stdout
f = open(ff_name, 'a',encoding='UTF-8')
sys.stdout = f

print("\n")
print("=" *50)
print("총 소요시간은 %s 초 이며," %round(t_time,1))
print("총 저장 건수는 %s 건 입니다 " %count)
print("=" *50)

sys.stdout = orig_stdout
f.close( )

#화면에 요약정보 출력하기
print("\n") 
print("=" *80)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 건수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)


#엑셀파일에 이미지 입력하기

excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
sheet.Columns(3).ColumnWidth = 30   #  이미지 가로 사이즈에 맞게 컬럼 크기 조정
row_cnt = cnt+1
sheet.Rows("2:%s" %row_cnt).RowHeight = 120  #  이미지 세로 사이즈에 맞게 로우 크기 조정

ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

for a in range(2,cnt+2) :
      col_name='C'+str(a)
      col_name2.append(col_name)

for b in range(1,cnt+1) :
      file_name=img_dir+'\\'+str(b)+'.jpg'
      file_name2.append(file_name)
      
for i in range(0,cnt) :
      rng = ws.Range(col_name2[i])
      image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100)
      excel.Visible=True
      excel.ActiveWorkbook.Save()
      
driver.close()


연습문제 9-5. 인터파크 쇼핑몰의 Best seller 상품 정보 추출하기
  1.크롤링 할 건수는 몇건입니까?: 20
  2.파일이 저장될 경로만 쓰세요;(예:c:\temp\): c:\data\


  요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요~~


----------------------------------------------------------------------
1.판매순위: 1
2.제품소개: 국내 정식발매 QCY T1 TWS 블루투스 무선 이어폰 5.0
3.판매가격: 18,240원                                    ~                  
4.혜택: 


----------------------------------------------------------------------
1.판매순위: 2
2.제품소개: [2,500P] 신세계백화점 모바일상품권_기프티쇼
3.판매가격: 100,000원
4.혜택: 무료배송


----------------------------------------------------------------------
1.판매순위: 3
2.제품소개: 샤오미 센서형  손 세정기 2세대
3.판매가격: 18,600원
4.혜택: 무료배송


----------------------------------------------------------------------
1.판매순위: 4
2.제품소개: [11382-14G/060] 크록스 여성 레이웨지샌들W 2종 택1(19SWCH205750)
3.판매가격: 35,000원
4.혜택: 무료배송


----------------------------------------------------------------------
1.판매순위: 5
2.제품소개: #쿠폰적용가 29,920원# 2019년형 하기스 에어솔솔 썸머팬티 4/5단계*2팩
3.판매가격: 37,400원
4.혜택: 무료배송


-----------------------------